# Simple Test

In [1]:
import networkx as nx
import numpy as np
# Test with sample graph in Fig 2.b
G = nx.Graph()
nodes = [1, 2, 3, 4, 5]
edges = [(1, 3), (3, 5), (3, 2), (3, 4), (2, 5), (2, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

L = nx.laplacian_matrix(G).toarray()
L_dagger = np.linalg.pinv(L)

n = G.number_of_nodes()
effective_resistance_sum = 0
for i in range(n):
    for j in range(n):
        b_xy = np.zeros(n)
        b_xy[i] = 1
        b_xy[j] = -1
        effective_resistance = np.dot(np.dot(b_xy.T, L_dagger), b_xy)
        if b_xy[1] and sum(b_xy) == 0:
            effective_resistance_sum += effective_resistance
            # print(b_xy, eff_edge)
effective_resistance_sum = 2 * len(nodes)/effective_resistance_sum
print(effective_resistance_sum)

1.5384615384615414


# Test when edge remove

In [2]:
# Test with sample graph in Fig 2.b
G = nx.Graph()
nodes = [1, 2, 3, 4, 5]
edges = [(1, 3), (3, 5), (3, 2), (3, 4), (2, 5), (2, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)
n = G.number_of_nodes()
G.remove_edges_from([(2, 4), (2, 5)])
L = nx.laplacian_matrix(G).toarray()
L_dagger = np.linalg.pinv(L)
effective_resistance_sum = 0
for i in range(n):
    for j in range(n):
        b_xy = np.zeros(n)
        b_xy[i] = 1
        b_xy[j] = -1
        effective_resistance = np.dot(np.dot(b_xy.T, L_dagger), b_xy)
        if b_xy[1]:
            effective_resistance_sum += effective_resistance
effective_resistance_sum = 2 * len(nodes)/effective_resistance_sum
print(effective_resistance_sum)

0.6775067750677507


# Calculate information centrality by [45]

In [3]:
import networkx as nx
import numpy as np

def compute_information_centrality_by_Laplacian(G: nx.Graph, node):
    L = nx.laplacian_matrix(G).toarray()
    n = G.number_of_nodes()
    L_dagger = np.linalg.pinv(L)
    effective_resistance_of_node = n * L_dagger[node][node] + np.trace(L_dagger)
    information_centrality = n / effective_resistance_of_node
    return information_centrality, effective_resistance_of_node


# Test with Fig.3(b)

In [4]:
# Test with sample graph in Fig 2.b
G = nx.Graph()
nodes = [0,1,2,3,4]
edges = [(0,2), (1,2), (1,3), (1, 4), (2, 3), (2, 4)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Compute the modified information centrality for node 2 in the original graph
initial_centrality, initial_effective = compute_information_centrality_by_Laplacian(G, 1)
print(f"Modified information centrality of node 2 in the original graph: {initial_centrality:.2f}")
# print(f"initial_effective: {initial_effective:.2f}")

# Remove edges (2, 4) and (2, 5), and compute the modified information centrality for node 2
G_removed = G.copy()
G_removed.remove_edges_from([(1, 3), (1, 4)])
centrality_removed_23_25, _ = compute_information_centrality_by_Laplacian(G_removed, 1)
print(f"Modified information centrality of node 2 after removing edges (2, 4) and (2, 5): {centrality_removed_23_25:.2f}")

# Remove edges (2, 3) and (2, 4), and compute the modified information centrality for node 2
G_removed = G.copy()
G_removed.remove_edges_from([(1, 2), (1, 3)])
centrality_removed_23_24, _ = compute_information_centrality_by_Laplacian(G_removed, 1)
print(f"Modified information centrality of node 2 after removing edges (2, 3) and (2, 4): {centrality_removed_23_24:.2f}")

Modified information centrality of node 2 in the original graph: 1.54
Modified information centrality of node 2 after removing edges (2, 4) and (2, 5): 0.71
Modified information centrality of node 2 after removing edges (2, 3) and (2, 4): 0.56


# OPTIMUM

In [5]:

def OPTIMUM(G:nx.Graph,T,k):
    P = []
    n = G.number_of_nodes()

    L = nx.laplacian_matrix(G).toarray()
    L_dagger = np.linalg.pinv(L)

    for _ in range(k):
        information_centrality_list = []
        for e in G.edges():
            information_centrality = 0
            information_centrality_sum = 0
            x, y = e
            G_copy = G.copy()
            G_copy.remove_edge(x, y)
            L = nx.laplacian_matrix(G_copy).toarray()
            L_dagger = np.linalg.pinv(L)
            for v in T:
                effective_resistance_of_node = n * L_dagger[v][v] + np.trace(L_dagger)
                information_centrality = n / effective_resistance_of_node
                information_centrality_sum += information_centrality
            information_centrality_list.append((information_centrality_sum / k, e))
        arg_min = min(information_centrality_list)
        P.append(arg_min[1])
        G.remove_edge(arg_min[1][0], arg_min[1][1])
    return P

# EXACTSM

In [6]:
def EXACTSM(G:nx.Graph,T,k):
    P = []
    n = G.number_of_nodes()
    L = nx.laplacian_matrix(G).toarray()
    L_dagger = np.linalg.pinv(L)

    for _ in range(k):
        information_centrality_list = []
        for e in G.edges():
            information_centrality = 0
            x, y = e
            G_copy = G.copy()
            G_copy.remove_edge(x, y)    
            if nx.is_connected(G_copy):
                b_e = np.zeros(n)
                b_e[x] = 1
                b_e[y] = -1
                # Calculate a, b, and c as defined in Lemma VII.1
                a = 1 - np.dot(b_e.T, np.dot(L_dagger, b_e))
                b = np.dot(b_e.T, np.dot(np.dot(L_dagger, L_dagger), b_e))
                L_dagger_b_e = np.dot(L_dagger, b_e)
                for v in T:
                    c = L_dagger_b_e[v] ** 2

                    # Calculate numerator and denominator
                    numerator = -(n * b + n**2 * c)
                    term1 = n * a * L_dagger[v][v] + n * c
                    term2 = a * np.trace(L_dagger) + b
                    term3 = n * L_dagger[v][v] + np.trace(L_dagger)
                    denominator = (term1 + term2) * term3

                    # Calculate information centrality
                    information_centrality += numerator / denominator

            information_centrality_list.append((information_centrality, e))
        arg_min = min(information_centrality_list)
        P.append(arg_min[1])
        G.remove_edge(arg_min[1][0], arg_min[1][1])
        b_ei = np.zeros(n)
        b_ei[arg_min[1][0]] = 1
        b_ei[arg_min[1][1]] = -1
        L_dagger += np.dot(np.dot(np.dot(L_dagger, b_ei), b_ei.T), L_dagger) / (1 - np.dot(np.dot(b_ei.T, L_dagger), b_ei))
    return P

# SETUP

In [12]:
# Read the file to RAM
# small graph: karate, polbooks, hamsterster, ca-GrQc
with open('./polbooks.mtx', 'r') as file:
    result = [(int(a) - 1, int(b) - 1) for a, b in (line.split() for line in file)]

In [13]:
import random
# Input to graph
G = nx.Graph()
edges = result
G.add_edges_from(edges)

# Experiment set
number_of_target_nodes = 10
k = 10

# Random sample
target_nodes = [1, 21, 25, 30, 44, 63, 73, 79, 95, 102]
# target_nodes = random.sample(list(G.nodes()), number_of_target_nodes)
all_information = 0
for i in target_nodes:
    initial_centrality, initial_effective = compute_information_centrality_by_Laplacian(G, i)
    # print(f"Information centrality of node {i}: {initial_centrality:.2f}")
    all_information += initial_centrality
print(f"Target nodes: {target_nodes}")
print(f"Information centrality average of target nodes: {all_information/number_of_target_nodes}")

Target nodes: [1, 21, 25, 30, 44, 63, 73, 79, 95, 102]
Information centrality average of target nodes: 2.3384766618919075


# TEST: Random select edges

In [14]:
G_copy = G.copy()
random_edges = random.sample(list(G_copy.edges()), k)
G_copy.remove_edges_from(random_edges)
# Test
all_information = 0
for i in target_nodes:
    initial_centrality, initial_effective = compute_information_centrality_by_Laplacian(G_copy, i)
    all_information += initial_centrality
print(f"Remove random edges: {random_edges}")
print(f"Target nodes: {target_nodes}")
print(f"Information centrality average of target nodes: {all_information / number_of_target_nodes}")

Remove random edges: [(73, 92), (19, 55), (27, 40), (74, 84), (58, 85), (88, 89), (75, 83), (102, 94), (30, 66), (12, 17)]
Target nodes: [1, 21, 25, 30, 44, 63, 73, 79, 95, 102]
Information centrality average of target nodes: 2.2503509813998326


# TEST: OPTIMUM

In [15]:
G_copy = G.copy()
P = OPTIMUM(G_copy, target_nodes, k)
print("Need remove edge:", P)
# Check if edge remove
G_copy = G.copy()
G_copy.remove_edges_from(P)
# Test
all_information = 0
for i in target_nodes:
    initial_centrality, initial_effective = compute_information_centrality_by_Laplacian(G_copy, i)
    # print(f"Information centrality of node {i}: {initial_centrality:.2f}")
    all_information += initial_centrality
print(f"Target nodes: {target_nodes}")
print(f"Information centrality average of target nodes: {all_information / number_of_target_nodes}")

Need remove edge: [(67, 103), (67, 104), (72, 92), (73, 92), (30, 67), (66, 67), (67, 64), (46, 102), (53, 76), (19, 77)]
Target nodes: [1, 21, 25, 30, 44, 63, 73, 79, 95, 102]
Information centrality average of target nodes: 1.8221472063961683


# TEST: EXACTSM

In [16]:
G_copy = G.copy()
P = EXACTSM(G_copy, target_nodes, k)
print("Need remove edge:", P)
# Check if edge remove
G_copy = G.copy()
G_copy.remove_edges_from(P)
# Test
all_information = 0
for i in target_nodes:
    initial_centrality, initial_effective = compute_information_centrality_by_Laplacian(G_copy, i)
    # print(f"Information centrality of node {i}: {initial_centrality:.2f}")
    all_information += initial_centrality
print(f"Target nodes: {target_nodes}")
print(f"Information centrality average of target nodes: {all_information / number_of_target_nodes}")

Need remove edge: [(100, 98), (30, 79), (46, 102), (74, 79), (79, 91), (79, 100), (72, 79), (75, 79), (71, 79), (87, 98)]
Target nodes: [1, 21, 25, 30, 44, 63, 73, 79, 95, 102]
Information centrality average of target nodes: 2.22715799274928


In [18]:
G_copy = G.copy()
target_nodes = [24, 26, 28, 32, 43, 44, 54, 95, 101, 104]
P = [(104, 69), (104, 67), (28, 4), (72, 28), (102, 46), (95, 73), (95, 61), (95, 94), (102, 93), (76, 53)]
print("Need remove edge:", P)
# Check if edge remove
G_copy = G.copy()
G_copy.remove_edges_from(P)
# Test
all_information = 0
for i in target_nodes:
    initial_centrality, initial_effective = compute_information_centrality_by_Laplacian(G_copy, i)
    # print(f"Information centrality of node {i}: {initial_centrality:.2f}")
    all_information += initial_centrality
print(f"Target nodes: {target_nodes}")
print(f"Information centrality average of target nodes: {all_information / number_of_target_nodes}")

Need remove edge: [(104, 69), (104, 67), (28, 4), (72, 28), (102, 46), (95, 73), (95, 61), (95, 94), (102, 93), (76, 53)]
Target nodes: [24, 26, 28, 32, 43, 44, 54, 95, 101, 104]
Information centrality average of target nodes: 1.8667036248343578
